The dataset contains numerous sets of content recommendations served to a specific user in a specific context. Each context (i.e. a set of recommendations) is given a display_id. In each such set, the user has clicked on at least one recommendation. The identities of the clicked recommendations in the test set are not revealed. Your task is to rank the recommendations in each group by decreasing predicted likelihood of being clicked.
As a warning, this is a very large relational dataset. While most of the tables are small enough to fit in memory, the page views log (page_views.csv) is over 2 billion rows and 100GB uncompressed. We have also uploaded a sample version of this file with the first 10,000,000 rows. The MD5 checksum of page_views.csv.zip is 3742c116bab4030e0a7ea1c0be623bd9.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

clicks_test.csv
clicks_train.csv
documents_categories.csv
documents_entities.csv
documents_meta.csv
documents_topics.csv
events.csv
page_views_sample.csv
promoted_content.csv
sample_submission.csv



In [2]:
import pandas as pd 
import numpy as np

In [3]:
ClicksTest=pd.read_csv("../input/clicks_test.csv",encoding="utf8")

In [4]:
ClicksTest.head()

,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754
3,16874594,170392
4,16874594,172888


In [5]:
ClicksTest.shape

(32225162, 2)

In [6]:
ClicksTrain=pd.read_csv("../input/clicks_train.csv",encoding="utf8")

In [7]:
ClicksTrain.head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [8]:
ClicksTrain.shape

(87141731, 3)

In [9]:
DocumentsCategories=pd.read_csv("../input/documents_categories.csv",encoding="utf8")

In [10]:
DocumentsCategories.head()

,document_id,category_id,confidence_level
0,1595802,1611,0.92
1,1595802,1610,0.07
2,1524246,1807,0.92
3,1524246,1608,0.07
4,1617787,1807,0.92


In [11]:
DocumentsCategories.shape

(5481475, 3)

In [12]:
DocumentsEntities=pd.read_csv("../input/documents_entities.csv",encoding="utf8")

In [13]:
DocumentsEntities.head()

,document_id,entity_id,confidence_level
0,1524246,f9eec25663db4cd83183f5c805186f16,0.672865
1,1524246,55ebcfbdaff1d6f60b3907151f38527a,0.399114
2,1524246,839907a972930b17b125eb0247898412,0.392096
3,1524246,04d8f9a1ad48f126d5806a9236872604,0.213996
4,1617787,612a1d17685a498aff4f036c1ee02c16,0.386193


In [14]:
DocumentsEntities.shape

(5537552, 3)

In [15]:
DocumentsMeta=pd.read_csv("../input/documents_meta.csv",encoding="utf8")

In [16]:
DocumentsMeta.head()

,document_id,source_id,publisher_id,publish_time
0,1595802,1.0,603.0,2016-06-05 00:00:00
1,1524246,1.0,603.0,2016-05-26 11:00:00
2,1617787,1.0,603.0,2016-05-27 00:00:00
3,1615583,1.0,603.0,2016-06-07 00:00:00
4,1615460,1.0,603.0,2016-06-20 00:00:00


In [17]:
DocumentsMeta.shape

(2999334, 4)

In [18]:
DocumentsTopics=pd.read_csv("../input/documents_topics.csv",encoding="utf8")

In [19]:
DocumentsTopics.head()

,document_id,topic_id,confidence_level
0,1595802,140,0.073113
1,1595802,16,0.059416
2,1595802,143,0.045421
3,1595802,170,0.038867
4,1524246,113,0.196450


In [20]:
DocumentsTopics.shape

(11325960, 3)

In [21]:
Events=pd.read_csv("../input/events.csv",encoding="utf8")

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
Events.head()

,display_id,uuid,document_id,timestamp,platform,geo_location
0,1,cb8c55702adb93,379743,61,3,US>SC>519
1,2,79a85fa78311b9,1794259,81,2,US>CA>807
2,3,822932ce3d8757,1179111,182,2,US>MI>505
3,4,85281d0a49f7ac,1777797,234,2,US>WV>564
4,5,8d0daef4bf5b56,252458,338,2,SG>00


In [23]:
Events.shape

(23120126, 6)

In [24]:
PageViewsSample=pd.read_csv("../input/page_views_sample.csv",encoding="utf8")

In [25]:
PageViewsSample.head()

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1
3,8205775c5387f9,120,44196592,1,IN>16,2
4,9cb0ccd8458371,120,65817371,1,US>CA>807,2


In [26]:
PageViewsSample.shape

(9999999, 6)

In [27]:
PromotedContent=pd.read_csv("../input/promoted_content.csv",encoding="utf8")

In [28]:
PromotedContent.head()

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7
3,4,471471,2,7
4,5,471472,2,7


In [29]:
PromotedContent.shape

(559583, 4)

We will use Merge to reduce the number of tables . the ideal is to get all the information in one table but we will try to perform this in sections



 1. Document Table